#패키지 불러오기 및 각종 설치 & 폰트 설정 한방에 



In [ ]:
#colab에서 pycaret 사용 시 해당 코드 실행 후 런타임 다시 시작으로 코드 실행해야 실행 가능
import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm  

!apt-get update -qq
!apt-get install fonts-nanum* -qq

font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()
matplotlib.rcParams['axes.unicode_minus'] = False

!pip install catboost
!pip install pycaret
from tqdm.notebook import tqdm
from pycaret.regression import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

NanumBarunGothic


In [ ]:
from google.colab import drive  # 구글 드라이브 마운트
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import pandas as pd 

train = pd.read_csv('/gdrive/My Drive/dacon/물류 예측/DataCh/train.csv') ##각 데이터 경로 반영
test = pd.read_csv('/gdrive/My Drive/dacon/물류 예측/DataCh/test.csv')

submission = pd.read_csv('/gdrive/My Drive/dacon/물류 예측/DataCh/sample_submission.csv')

In [ ]:
train.head() #train_data 상단 5개 데이터 확인

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,0,5011000595017300,2871000192069300,음반,3
1,1,4148000690043300,5011000264024400,문화컨텐츠,3
2,2,5011000078068400,1120000007005400,농산물,3
3,3,4127100048006400,5011000587019400,기타식품,7
4,4,5011000078068400,2823700010076300,농산물,3


In [ ]:
test.head()#test_data 상단 5개 데이터 확인

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
0,0,4167000577042200,5011000435014100,선케어
1,1,1156000009012200,5011000172034400,구강위생용품
2,2,4122000363057300,5011000361097300,캠핑
3,3,5011000436041400,2826000084036400,아웃도어가구
4,4,4150000241065200,5011000169044300,분유/이유식/아기간식


In [ ]:
train.info() #train_data null 값 확인 용, 해당 데이터의 경우 없네요 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31684 entries, 0 to 31683
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         31684 non-null  int64 
 1   송하인_격자공간고유번호  31684 non-null  int64 
 2   수하인_격자공간고유번호  31684 non-null  int64 
 3   물품_카테고리       31684 non-null  object
 4   운송장_건수        31684 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 1.2+ MB


In [ ]:
test.info() #test 역시 마찬가지 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7920 entries, 0 to 7919
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         7920 non-null   int64 
 1   송하인_격자공간고유번호  7920 non-null   int64 
 2   수하인_격자공간고유번호  7920 non-null   int64 
 3   물품_카테고리       7920 non-null   object
dtypes: int64(3), object(1)
memory usage: 247.6+ KB


In [ ]:
train.isnull().sum() #null 값 개수 확인 

index           0
송하인_격자공간고유번호    0
수하인_격자공간고유번호    0
물품_카테고리         0
운송장_건수          0
dtype: int64

## 데이터 전처리

In [ ]:
# 원핫 인코딩 
train_one = pd.get_dummies(train)
test_one = pd.get_dummies(test)

In [ ]:
train_one['송하인_격자공간고유번호'] = np.log1p(train_one['송하인_격자공간고유번호']) ##데이터 셋이 살짝 달라져서 로그가 아닌 정규화를 진행해주시거나 해당 부분을 날리는 게 더 좋아보였습니다. 
test_one['송하인_격자공간고유번호'] = np.log1p(test_one['송하인_격자공간고유번호'])  ##이전 데이터 셋도 생각해보니 해당 특성을 버릴 생각은 안 했었네요..? 
                                                                                    ##많은 방법이 있을 것 같긴 합니다. 다양한 실험을 추천드립니다. 
train_one['수하인_격자공간고유번호'] = np.log1p(train_one['수하인_격자공간고유번호'])
test_one['수하인_격자공간고유번호'] = np.log1p(test_one['수하인_격자공간고유번호'])

In [ ]:
train_one

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,운송장_건수,물품_카테고리_DIY자재/용품,물품_카테고리_PC,물품_카테고리_PC주변기기,물품_카테고리_가공식품,물품_카테고리_가방,물품_카테고리_건강식품,물품_카테고리_건강용품,물품_카테고리_게임기/타이틀,물품_카테고리_계절가전,물품_카테고리_골프,물품_카테고리_공구,물품_카테고리_과자,물품_카테고리_구강위생용품,물품_카테고리_기능성,물품_카테고리_기저귀/물티슈,물품_카테고리_기타디지털/가전,물품_카테고리_기타스포츠/레저,물품_카테고리_기타식품,물품_카테고리_기타출산/육아,물품_카테고리_기타패션의류,물품_카테고리_기타패션잡화,물품_카테고리_기타화장품/미용,물품_카테고리_김치,물품_카테고리_낚시,물품_카테고리_남성화장품,물품_카테고리_냉동/간편조리식품,물품_카테고리_네일케어,물품_카테고리_농산물,물품_카테고리_눈관리용품,물품_카테고리_다이어트식품,물품_카테고리_등산,물품_카테고리_모니터,물품_카테고리_모자,물품_카테고리_문구/사무용품,물품_카테고리_문화컨텐츠,물품_카테고리_바디케어,...,물품_카테고리_언더웨어,물품_카테고리_완구/매트,물품_카테고리_욕실용품,물품_카테고리_원피스/점프슈트,물품_카테고리_위생/건강용품,물품_카테고리_유아가구,물품_카테고리_음료,물품_카테고리_음반,물품_카테고리_음향가전,물품_카테고리_의료용품,물품_카테고리_이미용가전,물품_카테고리_인테리어소품,물품_카테고리_자동차용품,물품_카테고리_잠옷,물품_카테고리_재활운동용품,물품_카테고리_주방가구,물품_카테고리_주방가전,물품_카테고리_주방용품,물품_카테고리_주얼리,물품_카테고리_지갑,물품_카테고리_축산,물품_카테고리_출산/유아동의류,물품_카테고리_출산/유아동잡화,물품_카테고리_취미용품,물품_카테고리_침구단품,물품_카테고리_침구세트,물품_카테고리_침실가구,물품_카테고리_카페트/러그,물품_카테고리_캠핑,물품_카테고리_커튼/블라인드,물품_카테고리_클렌징,물품_카테고리_태블릿PC/노트북액세서리,물품_카테고리_패션소품,물품_카테고리_하의,물품_카테고리_향수,물품_카테고리_헤어스타일링,물품_카테고리_헤어액세서리,물품_카테고리_헤어케어,물품_카테고리_헬스,물품_카테고리_홈데코
0,0,36.150412,35.593437,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,35.961403,36.150412,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,36.150412,34.652105,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,35.956351,36.150412,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,36.150412,35.576824,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31679,31679,36.036389,36.150412,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31680,31680,34.660109,36.150412,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31681,31681,34.660109,36.150412,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31682,31682,35.956400,36.150412,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_one

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리_DIY자재/용품,물품_카테고리_PC,물품_카테고리_PC주변기기,물품_카테고리_가공식품,물품_카테고리_가방,물품_카테고리_건강식품,물품_카테고리_건강용품,물품_카테고리_게임기/타이틀,물품_카테고리_계절가전,물품_카테고리_골프,물품_카테고리_공구,물품_카테고리_과자,물품_카테고리_구강위생용품,물품_카테고리_기능성,물품_카테고리_기저귀/물티슈,물품_카테고리_기타디지털/가전,물품_카테고리_기타스포츠/레저,물품_카테고리_기타식품,물품_카테고리_기타출산/육아,물품_카테고리_기타패션의류,물품_카테고리_기타패션잡화,물품_카테고리_기타화장품/미용,물품_카테고리_김치,물품_카테고리_낚시,물품_카테고리_남성화장품,물품_카테고리_냉동/간편조리식품,물품_카테고리_네일케어,물품_카테고리_농산물,물품_카테고리_눈관리용품,물품_카테고리_다이어트식품,물품_카테고리_등산,물품_카테고리_모니터,물품_카테고리_모자,물품_카테고리_문구/사무용품,물품_카테고리_문화컨텐츠,물품_카테고리_바디케어,물품_카테고리_반려동물,...,물품_카테고리_언더웨어,물품_카테고리_완구/매트,물품_카테고리_욕실용품,물품_카테고리_원피스/점프슈트,물품_카테고리_위생/건강용품,물품_카테고리_유아가구,물품_카테고리_음료,물품_카테고리_음반,물품_카테고리_음향가전,물품_카테고리_의료용품,물품_카테고리_이미용가전,물품_카테고리_인테리어소품,물품_카테고리_자동차용품,물품_카테고리_잠옷,물품_카테고리_재활운동용품,물품_카테고리_주방가구,물품_카테고리_주방가전,물품_카테고리_주방용품,물품_카테고리_주얼리,물품_카테고리_지갑,물품_카테고리_축산,물품_카테고리_출산/유아동의류,물품_카테고리_출산/유아동잡화,물품_카테고리_취미용품,물품_카테고리_침구단품,물품_카테고리_침구세트,물품_카테고리_침실가구,물품_카테고리_카페트/러그,물품_카테고리_캠핑,물품_카테고리_커튼/블라인드,물품_카테고리_클렌징,물품_카테고리_태블릿PC/노트북액세서리,물품_카테고리_패션소품,물품_카테고리_하의,물품_카테고리_향수,물품_카테고리_헤어스타일링,물품_카테고리_헤어액세서리,물품_카테고리_헤어케어,물품_카테고리_헬스,물품_카테고리_홈데코
0,0,35.965973,36.150412,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,34.683742,36.150412,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,35.955115,36.150412,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,3,36.150412,35.577639,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,35.961885,36.150412,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,7915,36.150412,36.069820,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7916,7916,34.682444,36.150412,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7917,7917,36.150811,34.675926,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7918,7918,36.150811,35.675647,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


상관계수 값 확인 plot을 그리려고 했으나 너무 낮은 값들이 분포해 있는 관계로 의미 없다고 생각했습니다. 

In [ ]:
train_one.corr()['운송장_건수'].sort_values()  

물품_카테고리_농산물        -0.110292
송하인_격자공간고유번호       -0.035819
물품_카테고리_상의         -0.016785
물품_카테고리_하의         -0.016054
물품_카테고리_음반         -0.015856
                      ...   
물품_카테고리_기타화장품/미용    0.050576
수하인_격자공간고유번호        0.052448
물품_카테고리_축산          0.064814
물품_카테고리_기타패션의류      0.102709
운송장_건수              1.000000
Name: 운송장_건수, Length: 104, dtype: float64

## Pycaret 모델링

[주차 수요 예측 경진대회](https://dacon.io/codeshare/3015) 때 진행했던 pycaret 세팅으로 동일하게 진행했습니다. 단순히 어떤 모델, boost 계열이 성능이 잘 뽑히는지 궁금해서 진행한 것입니다. 

In [ ]:
reg = setup(data=train_one,
            target='운송장_건수',
            session_id = 201,
            numeric_imputation = 'mean',
            fold_shuffle = True,
            numeric_features=list(train_one.drop(columns = ['운송장_건수', 'index']).columns),
            ignore_low_variance = True,
            combine_rare_levels = True, rare_level_threshold = 0.05,
            remove_multicollinearity = True, multicollinearity_threshold = 0.90,
            normalize = True,
            silent= True)

,Description,Value
0,session_id,201
1,Target,운송장_건수
2,Original Data,"(31684, 104)"
3,Missing Values,False
4,Numeric Features,103
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(22178, 3)"


In [ ]:
best_3_l = compare_models(sort='RMSE', n_select=3) #RMSE 기준 상위 3개 모델 추출 

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,2.3941,44.6490,6.4894,-0.0003,0.4346,0.4554,0.530
catboost,CatBoost Regressor,2.4097,44.8173,6.5232,-0.0211,0.4411,0.4562,4.773
br,Bayesian Ridge,2.4287,47.3197,6.5772,0.0120,0.4388,0.4623,0.019
lr,Linear Regression,2.4288,47.3198,6.5773,0.0120,0.4390,0.4623,0.358
ridge,Ridge Regression,2.4288,47.3198,6.5773,0.0120,0.4390,0.4623,0.016
lar,Least Angle Regression,2.4288,47.3198,6.5773,0.0120,0.4390,0.4623,0.018
omp,Orthogonal Matching Pursuit,2.4331,47.3485,6.5790,0.0115,0.4392,0.4633,0.019
lightgbm,Light Gradient Boosting Machine,2.4319,46.2560,6.5836,-0.0209,0.4445,0.4624,0.158
en,Elastic Net,2.4358,47.6847,6.6023,0.0048,0.4386,0.4649,0.019
lasso,Lasso Regression,2.4437,47.8968,6.6189,-0.0008,0.4416,0.4668,0.018


In [ ]:
blended_l = blend_models(estimator_list= best_3_l, fold=5, optimize='RMSE') ## 추출한 상위 3가지 모델 블랜딩
pred_holdout = predict_model(blended_l)
final_model_l = finalize_model(blended_l)
pred_esb_l = predict_model(final_model_l, test_one) 

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2.3639,28.6479,5.3524,-0.0556,0.4375,0.4699
1,2.5589,77.5968,8.8089,0.1009,0.4448,0.4502
2,2.4712,57.0749,7.5548,0.1344,0.4343,0.4533
3,2.3556,36.5521,6.0458,-0.0063,0.4323,0.4574
4,2.2389,23.7854,4.8770,-0.0053,0.4231,0.4485
Mean,2.3977,44.7314,6.5278,0.0337,0.4344,0.4559
SD,0.1091,19.9841,1.4558,0.0718,0.0071,0.0077


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,2.3429,38.7413,6.2243,0.0412,0.4281,0.4481


In [ ]:
submission.head()

,index,운송장_건수
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
submission['운송장_건수'] = pred_esb_l['Label'] # 예측결과 submission양식에 맞게 만들기 
submission

,index,운송장_건수
0,0,5.459672
1,1,5.548277
2,2,5.611683
3,3,5.853885
4,4,5.822540
...,...,...
7915,7915,4.534412
7916,7916,5.326877
7917,7917,4.168127
7918,7918,4.266920


In [ ]:
submission.to_csv('/gdrive/My Drive/dacon/물류 예측/제출용/index_drop_dtccode.csv' ,index = False) #제출 파일 만들기, 원하시는 위치로 경로 설정 

#GradientBoostingRegressor
Pycaret 돌렸을 때 상위 모델이어서 각각 튜닝하고 앙상블 할 생각으로 모델을 만들기 시작했지만 시험기간인 관계로 중단했던 상황이었습니다...
대회 진행했던 데이터나 현재 데이터나 GBR 모델이 가장 성능이 잘 나왔습니다. 

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor ##Pycaret 돌렸을 때 상위 모델이어서 각각 튜닝하고 앙상블 할 생각으로
                                                      ##모델을 만들기 시작했지만 시험기간인 관계로 중단했던 상황... 
train_X = train_one.drop(['index', '운송장_건수'] ,axis = 1) ##대회 진행했던 데이터나 현재 데이터나 GBR 모델이 가장 성능이 잘 나옴
train_Y = train_one['운송장_건수']

#모델 정의
gbr = GradientBoostingRegressor()
gbr.fit(train_X, train_Y)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
test_one = test_one.drop('index', axis=1)

In [ ]:
pred = gbr.predict(test_one)

In [ ]:
submission['운송장_건수'] = pred

In [ ]:
submission.to_csv('/gdrive/My Drive/dacon/물류 예측/제출용/gbr_dtccode.csv',index = False)